In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from signals import *
from indicators.indicator.indicators import SuperTrend
from hyperopt import hp
from hyperopt import fmin, tpe
signal_file = "./telegram/2020-30-01_TelegramDump.json"

In [2]:
# How does indexing work in a resampled df
# Signal comes in at 03:30, so the 03:00 candle has not finished yet, we should use the 
# indicator from the previous candle, 02:00
#display(pd.concat([df_60.loc[:"2019-10-29 03:30"][-2:],df_60.loc["2019-10-29 03:30":]])) # -> Starts at 04:00, is this what I want

def trailing_df(df, datetime_str):
    """
    Given a signal within a candle, this function returns a df starting with the most recent 
    complete candle.
    
    e.g.
    1 hr sampling. 03:30 signal time (3-4 candle not yet complete), returns starting with the 02:00 candle
    """  
    return pd.concat([df.loc[:datetime_str][-2:],df.loc[datetime_str:]])
    

def supertrend_trailing(df_full, signal, st_name, st_type):
    """
    Process signal using trailing supertrend stop loss.
    TODO: accound for spread and fees
    """
    
    fees = 3.5 # pips, approximate
    df = trailing_df(df_full, signal.datetime_str)

    real_entry = signal.real_entry_price
    stop_loss = signal.stop_loss
    
    ohlc_values = df.iterrows()
    _, previous_ohlc = next(ohlc_values)
    
    previous_type = previous_ohlc[st_type]
    previous_supertrend = previous_ohlc[st_name]
    

    for dt, ohlc in ohlc_values:
        if signal.type == "BUY":
            if previous_type == 1:
                if stop_loss < previous_supertrend:
                    stop_loss = previous_supertrend
            
            if ohlc.High > signal.take_profit:
                return signal.take_profit - real_entry - fees
            if ohlc.Low < stop_loss:
                return stop_loss - real_entry - fees
            
        if signal.type == "SELL":
            if previous_type == -1:
                if stop_loss > previous_supertrend:
                    stop_loss = previous_supertrend   
                    
            if ohlc.Low < signal.take_profit:
                return real_entry - signal.take_profit - fees
            if ohlc.High > stop_loss:
                return real_entry - stop_loss - fees
            
        previous_type = previous_ohlc[st_type]
        previous_supertrend = previous_ohlc[st_name]
        
    return 0.0
        

In [6]:
# Single test trade
beta_tp1 = Signal(ticker="NZDCAD", datetime_str="2019-12-04 12:44",
                  entry_price=0.8656, stop_loss=0.868, take_profit=0.8636)
period = 40
multiplier = 6
df = load_ticker(beta_tp1.ticker, 60)
SuperTrend(df, period, multiplier)
st_name = 'ST_' + str(period) + '_' + str(multiplier)
st_type = 'STX_' + str(period) + '_' + str(multiplier)

print(beta_tp1, supertrend_trailing(df, beta_tp1, st_name, st_type) * pip_factor(beta_tp1.ticker))

NZDCAD: 2019-12-04 12:44: Entry: 0.86560, SL: 0.86800, TP: 0.86360 20.000000000000018


In [18]:
# Load all trades with one set of parameters
multiplier=5
period=62

signals = load_beta_trades(signal_file)
total_profit = 0.0
wins = 0
losses = 0
balance = 1000.0

for signal in signals[-100:]:
    #signal.set_stop_loss_pips(11)
    signal.set_take_profit_pips(100)
    df = load_ticker(signal.ticker, 60)
    SuperTrend(df, period, multiplier)
    st_name = 'ST_' + str(period) + '_' + str(multiplier)
    st_type = 'STX_' + str(period) + '_' + str(multiplier)
    profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
    balance += profit * (balance / 1000)
    
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    #print(signal, profit)
print(f"Profit: {total_profit} pips, Balance: {balance}, Wins: {wins}, Losses: {losses}")

Profit: 1554.3278108461654 pips, Balance: 3655.6808109199087, Wins: 41, Losses: 59


With SL, TP and Trailing loss
60mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 6
profit = 1514 pips (30 wins, 22 losses)

30mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 8
profit = 1430 pips (29 wins, 23 losses)

15mins
SL = 50 pips
TP = 70 pips
Length = 40
Multiplier = 13
profit = 1227 pips (27 wins, 25 losses)

In [15]:
# Bayesian analysis
def objective(params):
    frequency = params['frequency']
    multiplier = int(params['multiplier'])
    period = int(params['period'])
    #sl = params['stop_loss']
    tp = params['take_profit']
    
    
    signals = load_beta_trades(signal_file)
    total_profit = 0.0
    profits = []
    wins = 0
    losses = 0
    balance = 1000.0

    for signal in signals[-100:]:
        #signal.set_stop_loss_pips(sl)
        signal.set_take_profit_pips(tp)
        df = load_ticker(signal.ticker, frequency)
        SuperTrend(df, period, multiplier)
        st_name = 'ST_' + str(period) + '_' + str(multiplier)
        st_type = 'STX_' + str(period) + '_' + str(multiplier)
        profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        profits.append(profit)
        balance += profit * (balance / 1000)

        # print(signal, profit)
    #return -np.mean(profits)/np.std(profits) # Sharpe Ratio
    #return -total_profit
    return -balance

space = {'frequency': hp.choice('frequency', [60, 240]),
         'multiplier': hp.quniform('multiplier', 1, 10, 1),
         'period': hp.quniform('period', 10, 100, 1),
         'take_profit': hp.quniform('take_profit', 10, 200, 1),
         #'stop_loss': hp.quniform('stop_loss', 10, 100, 1),
        }

for _ in range(5):
    print(fmin(objective, space, algo=tpe.suggest, max_evals=32))


100%|██████████| 32/32 [06:37<00:00, 12.41s/trial, best loss: -4053.4601144658286]
{'frequency': 0, 'multiplier': 6.0, 'period': 34.0, 'take_profit': 108.0}
100%|██████████| 32/32 [06:08<00:00, 11.51s/trial, best loss: -4344.630552790772] 
{'frequency': 0, 'multiplier': 5.0, 'period': 62.0, 'take_profit': 111.0}
 59%|█████▉    | 19/32 [02:04<01:25,  6.55s/trial, best loss: -3088.5695304487617]


KeyboardInterrupt: 

### Current Setup
1858 pips 56 wins 44 losses
{'frequency': 60, 'multiplier': 6, 'period': 40, 'take_profit': 69.0}

### Maximum Profit
100%|██████████| 256/256 [1:05:17<00:00, 15.30s/trial, best loss: -2023.6072397059888]
{'frequency': 60, 'multiplier': 5.0, 'period': 81.0, 'take_profit': 113.0}

### Maximum Sharpe Ratio
100%|██████████| 256/256 [28:59<00:00,  6.79s/trial, best loss: -0.5600357889275049]
{'frequency': 1, 'multiplier': 4.0, 'period': 19.0, 'take_profit': 10.0}


In [ ]:
# Multiplier Analysis

period = 40
sample_mins = 30

for multiplier in range(1, 13, 1):
    signals = load_beta_trades(signal_file)
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        #signal.set_stop_loss_pips(50)
        signal.set_take_profit_pips(69)

        df = get_supertrend(signal.ticker, multiplier=multiplier, period=period, sample_mins=sample_mins)
        st_name = 'ST_' + str(period) + '_' + str(multiplier)
        st_type = 'STX_' + str(period) + '_' + str(multiplier)
        profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        # print(signal, profit)
    print(multiplier, total_profit, wins, losses)

In [ ]:
# Length Analysis

multiplier = 8
sample_mins = 30
take_profit = 69


for period in range(10, 101, 10):
    signals = load_beta_trades(signal_file)
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        #signal.set_stop_loss_pips(42)
        signal.set_take_profit_pips(take_profit)
        df = get_supertrend(signal.ticker, multiplier=multiplier, period=period, sample_mins=sample_mins)
        st_name = 'ST_' + str(period) + '_' + str(multiplier)
        st_type = 'STX_' + str(period) + '_' + str(multiplier)
        profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    print(period, total_profit, wins, losses)

In [ ]:
# Stop Loss Analysis

multiplier = 6
period = 40
take_profit = 69

profits = []
stops = range(10, 100, 1)

for stop in stops:
    signals = load_beta_trades(signal_file)
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)

        df = get_supertrend(signal.ticker, multiplier=multiplier, period=period, sample_mins=60)      
        st_name = 'ST_' + str(period) + '_' + str(multiplier)
        st_type = 'STX_' + str(period) + '_' + str(multiplier)
        profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title=f"BETA: Stop Loss Optimisation (TP = {take_profit}, Mul = {multiplier}, Range = {period})",
    title_x=0.5,
    xaxis_title="Stop Loss (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Take Profit Analysis

multiplier = 6
period = 40
stop = 43

profits = []
take_profits = range(10, 100, 1)

for take_profit in take_profits:
    signals = load_beta_trades(signal_file)
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals[-100:]:
        #signal.set_stop_loss_pips(stop) # Using Betas stop losses
        signal.set_take_profit_pips(take_profit)

        df = get_supertrend(signal.ticker, multiplier=multiplier, period=period, sample_mins=60)
        st_name = 'ST_' + str(period) + '_' + str(multiplier)
        st_type = 'STX_' + str(period) + '_' + str(multiplier)
        profit = supertrend_trailing(df, signal, st_name, st_type) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title=f"BETA: Take Profit Optimisation (SL = {stop}, Mul = {multiplier}, Range = {length})",
    title_x=0.5,
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly scatter plot
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title="BETA Take Profit Optimisation",
    title_x=0.5,
    xaxis_title="Take Profit Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly candlestick plot
dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()

In [ ]:
# How to pretty print a dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_60)